In [37]:
import pygame
import os

# ---------------- Song Class ----------------
class Song:
    def __init__(self, title, artist, file_path):
        self.title = title
        self.artist = artist
        self.file_path = file_path
        self.next_song = None

    def __str__(self):
        return f"{self.title} by {self.artist}"


# ---------------- MusicPlaylist Class ----------------
class MusicPlaylist:
    def __init__(self):
        # pygame.mixer.init()                 # << Removed for Colab compatibility
        self.head = None
        self.current_song = None
        self.length = 0

    # ---------- Add Song (Upload เพลง) ----------
    def add_song(self, title, artist, file_path):
        if not os.path.exists(file_path):   # << เพิ่ม
            print("File not found.")
            return

        new_song = Song(title, artist, file_path)

        if self.head is None:
            self.head = new_song
            self.current_song = new_song
        else:
            current = self.head
            while current.next_song:
                current = current.next_song
            current.next_song = new_song

        self.length += 1
        print(f"Added: {new_song}")

    # ---------- Display Playlist ----------
    def display_playlist(self):
        if self.head is None:
            print("Playlist is empty.")
            return

        current = self.head
        print("\n--- Your Music Playlist ---")
        count = 1
        while current:
            print(f"{count}. {current}")
            current = current.next_song
            count += 1
        print("---------------------------")

    # ---------- Play Current Song ----------
    def play_current_song(self):
        if self.current_song:
            # pygame.mixer.music.load(self.current_song.file_path)  # << Removed for Colab compatibility
            # pygame.mixer.music.play()                             # << Removed for Colab compatibility
            print(f"\nNow playing (simulated): {self.current_song} (Audio playback not supported in this environment.)")
        else:
            print("Playlist is empty or no song is selected.")

    # ---------- Next Song ----------
    def next_song(self):
        if self.current_song and self.current_song.next_song:
            self.current_song = self.current_song.next_song
            self.play_current_song()
        elif self.current_song:
            print("End of playlist. No next song.")
        else:
            print("Playlist is empty.")

    # ---------- Previous Song ----------
    def prev_song(self):
        if self.head is None or self.current_song is None:
            print("Playlist is empty or no song is selected.")
            return

        if self.current_song == self.head:
            print("Already at the beginning of the playlist.")
            return

        current = self.head
        while current.next_song != self.current_song:
            current = current.next_song

        self.current_song = current
        self.play_current_song()

    # ---------- Pause / Resume / Stop ----------
    def pause(self):
        # pygame.mixer.music.pause()
        print("Paused (Audio playback not supported in this environment.)")

    def resume(self):
        # pygame.mixer.music.unpause()
        print("Resumed (Audio playback not supported in this environment.)")

    def stop(self):
        # pygame.mixer.music.stop()
        print("Stopped (Audio playback not supported in this environment.)")

    # ---------- Length ----------
    def get_length(self):
        return self.length

    # ---------- Delete Song ----------
    def delete_song(self, title):
        if self.head is None:
            print(f"Cannot delete '{title}'. Playlist is empty.")
            return

        if self.head.title == title:
            if self.current_song == self.head:
                self.current_song = self.head.next_song
            self.head = self.head.next_song
            self.length -= 1
            print(f"Deleted: {title}")
            return

        current = self.head
        prev = None
        while current and current.title != title:
            prev = current
            current = current.next_song

        if current:
            if self.current_song == current:
                self.current_song = current.next_song or prev

            prev.next_song = current.next_song
            self.length -= 1
            print(f"Deleted: {title}")
        else:
            print(f"Song '{title}' not found in the playlist.")

#สร้าง Web App

1. installing **streamlit** and **pyngrok** libraries

In [38]:
pip install streamlit pyngrok

2. สร้างไฟล์ .py

In [39]:
%%writefile app.py
import streamlit as st
# import pygame # Removed for Colab compatibility
import tempfile
import os

# ----------------- Init pygame -----------------
# pygame.mixer.init() # Removed for Colab compatibility

# ----------------- Song Class -----------------
class Song:
    def __init__(self, title, artist, file_path):
        self.title = title
        self.artist = artist
        self.file_path = file_path
        self.next_song = None

    def __str__(self):
        return f"{self.title} by {self.artist}"

# ----------------- MusicPlaylist Class -----------------
class MusicPlaylist:
    def __init__(self):
        self.head = None
        self.current_song = None
        self.length = 0

    def add_song(self, title, artist, file_path):
        new_song = Song(title, artist, file_path)

        if self.head is None:
            self.head = new_song
            self.current_song = new_song
        else:
            current = self.head
            while current.next_song:
                current = current.next_song
            current.next_song = new_song

        self.length += 1
        st.success(f"Added: {new_song}")

    def display_playlist(self):
        songs = []
        current = self.head
        i = 1
        while current:
            songs.append(f"{i}. {current.title} by {current.artist}")
            current = current.next_song
            i += 1
        return songs

    # Removed play_current_song, pause, resume, stop methods

    def next_song(self):
        if self.current_song and self.current_song.next_song:
            self.current_song = self.current_song.next_song
        elif self.current_song:
            st.warning("End of playlist.")
        else:
            st.warning("Playlist is empty.")

    def prev_song(self):
        if self.current_song == self.head or self.head is None:
            st.warning("Already at first song or playlist is empty.")
            return

        current = self.head
        prev = None
        while current and current != self.current_song:
            prev = current
            current = current.next_song

        if prev:
            self.current_song = prev
        else:
            st.warning("Already at first song.")

    def delete_song(self, title):
        if not self.head:
            st.error("Playlist is empty.")
            return

        if self.head.title == title:
            if self.current_song == self.head:
                self.current_song = self.head.next_song
            self.head = self.head.next_song
            self.length -= 1
            st.success(f"Deleted: {title}")
            return

        prev, current = None, self.head
        while current and current.title != title:
            prev = current
            current = current.next_song

        if current:
            if self.current_song == current:
                self.current_song = current.next_song if current.next_song else prev

            prev.next_song = current.next_song
            self.length -= 1
            st.success(f"Deleted: {title}")
        else:
            st.error("Song not found.")

    def get_length(self):
        return self.length

# ----------------- Streamlit App -----------------
st.title("🎶 Music Playlist App (Playable)")

if "playlist" not in st.session_state:
    st.session_state.playlist = MusicPlaylist()

# ----------------- Sidebar: Upload Song -----------------
st.sidebar.header("➕ Add New Song")

title = st.sidebar.text_input("Title")
artist = st.sidebar.text_input("Artist")
uploaded_file = st.sidebar.file_uploader(
    "Upload Song File", type=["mp3", "wav"]
)

if st.sidebar.button("Add Song"):
    if title and artist and uploaded_file:
        # Create a temporary file to store the uploaded audio
        with tempfile.NamedTemporaryFile(delete=False, suffix="." + uploaded_file.type.split('/')[-1]) as tmp_file:
            tmp_file.write(uploaded_file.read())
            temp_file_path = tmp_file.name

        st.session_state.playlist.add_song(
            title, artist, temp_file_path
        )
        # Clean up temporary files associated with deleted songs later, if needed.
    else:
        st.sidebar.warning("Please fill all fields and upload a song file.")

# ----------------- Sidebar: Delete Song -----------------
st.sidebar.markdown("---")
delete_title = st.sidebar.text_input("Delete Song by Title")
if st.sidebar.button("Delete Song"):
    st.session_state.playlist.delete_song(delete_title)

# ----------------- Playlist Display -----------------
st.header("📃 Current Playlist")
playlist_items = st.session_state.playlist.display_playlist()
if playlist_items:
    for song in playlist_items:
        st.write(song)
else:
    st.write("Playlist is empty.")

# ----------------- Playback Controls -----------------
st.markdown("---")
st.header("🎧 Playback Controls")

col1, col2, col3 = st.columns(3)

with col1:
    if st.button("⏪ Previous"):
        st.session_state.playlist.prev_song()

with col2:
    if st.button("▶️ Play/Change Song"):
        if st.session_state.playlist.current_song:
            file_path = st.session_state.playlist.current_song.file_path
            # Determine format based on file extension, assuming common audio types
            file_extension = os.path.splitext(file_path)[1].lower()
            if file_extension == '.mp3':
                audio_format = 'audio/mp3'
            elif file_extension == '.wav':
                audio_format = 'audio/wav'
            else:
                audio_format = 'audio/mpeg' # Default to mpeg for broader support

            st.audio(file_path, format=audio_format, start_time=0)
            st.info(f"Now playing: {st.session_state.playlist.current_song}")
        else:
            st.warning("No song selected or playlist is empty.")

with col3:
    if st.button("⏩ Next"):
        st.session_state.playlist.next_song()

# Display current song explicitly (optional, for better UI feedback)
if st.session_state.playlist.current_song and 'play_button_pressed' in st.session_state and st.session_state.play_button_pressed:
    st.write(f"Currently selected: {st.session_state.playlist.current_song}")

st.markdown("---")
st.write(f"🎼 Total songs: {st.session_state.playlist.get_length()}")

Overwriting app.py


3. Login **ngrok** ที่ https://ngrok.com/ and copy your **authtoken**

In [40]:
!ngrok authtoken 36bsLRji0VHPPLMcEZGBzSGHuvv_9M2xcPA5LhubnnEzodvG

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [41]:
from pyngrok import ngrok

ngrok.kill()  # ปิด tunnel เก่า

public_url = ngrok.connect(8501)
print("🌍 Open your app here:", public_url)

🌍 Open your app here: NgrokTunnel: "https://egregious-subconchoidal-alijah.ngrok-free.dev" -> "http://localhost:8501"


4. **รัน app Streamlit** ที่สร้างขึ้นมา
การบอกให้ Colab รันไฟล์ calculator.py ด้วย Streamlit บนพอร์ต 8501

In [ ]:
!streamlit run app.py --server.port 8501 &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.37.126:8501

